# Phase 2: Intermediate - Active Trading & Advisory

In this phase, we evolve from simple sequential pipelines to **Parallel Agent Execution**. This allows us to gather data from multiple independent markets (Equities and Crypto) simultaneously before synthesizing a recommendation.

### System Architecture
We use a branching flow where data collection happens in parallel:
```
          ┌→ Market Data Provider →┐
START ----┤                        ├→ Recommender Agent → END
          └→ Binance Tracker ------→┘
```

| Agent | Role | Parallel Status |
|---|---|---|
| **MarketDataProvider** | Analyzes traditional stock equity (e.g., NVDA) | Yes |
| **BinanceTracker** | Analyzed crypto assets (e.g., BTC-USD) | Yes |
| **RecommenderAgent** | Synthesizes a multi-market strategy | No (Wait for both) |

## 1. Environment Setup

In [1]:
%pip install vinagent==0.0.6.post3

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv('.env'))

llm = ChatOpenAI(model="gpt-4o-mini")
print("LLM initialized.")

Note: you may need to restart the kernel to use updated packages.
LLM initialized.


## 2. Define Multi-Market State

The state now contains specific keys for each market stream.

In [2]:
from typing import Annotated, TypedDict

def append_messages(existing: list, update: dict) -> list:
    return existing + [update]

class TradingState(TypedDict):
    messages: Annotated[list[dict], append_messages]
    equity_insight: str
    crypto_insight: str
    final_recommendation: str

In [3]:
from datetime import datetime
from vinagent.register import primary_function

@primary_function
def get_current_time() -> str:
    """
    Get the current date and time. Use this to know 'today's' date.
    Returns:
        str: Current date and time in YYYY-MM-DD HH:MM:SS format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Custom time tool defined.")

import requests
# You need to have your CoinDesk API in your environment variables
@primary_function
def fetch_btc_market_data() -> str:
    """
    Fetch comprehensive Bitcoin (BTC) market data including price, changes (daily/weekly/monthly), and moving averages from the CoinDesk API.
    Returns:
        str: A comprehensive summary of BTC market metrics.
    """
    try:
        response = requests.get('https://data-api.coindesk.com/index/cc/v1/latest/tick',
                params={"market":"cadli","instruments":"BTC-USD","apply_mapping":"true"},
                headers={"Content-type":"application/json; charset=UTF-8"}
        )
        data = response.json()['Data']['BTC-USD']
        
        summary = (
            f"--- BTC-USD Market Data ---\n"
            f"Current Price: ${data['VALUE']:.2f} USD\n"
            f"Day Change: {data['CURRENT_DAY_CHANGE']:.2f} ({data['CURRENT_DAY_CHANGE_PERCENTAGE']:.2f}%)\n"
            f"Week Change: {data['CURRENT_WEEK_CHANGE']:.2f} ({data['CURRENT_WEEK_CHANGE_PERCENTAGE']:.2f}%)\n"
            f"Month Change: {data['CURRENT_MONTH_CHANGE']:.2f} ({data['CURRENT_MONTH_CHANGE_PERCENTAGE']:.2f}%)\n"
            f"24h Range: ${data['MOVING_24_HOUR_LOW']:.2f} - ${data['MOVING_24_HOUR_HIGH']:.2f}\n"
            f"Year Change: {data['CURRENT_YEAR_CHANGE_PERCENTAGE']:.2f}%\n"
            f"Volatility Context: Day High ${data['CURRENT_DAY_HIGH']:.2f} / Low ${data['CURRENT_DAY_LOW']:.2f}"
        )
        return summary
    except Exception as e:
        return f"Error fetching BTC price: {e}"

print(fetch_btc_market_data())

Custom time tool defined.
--- BTC-USD Market Data ---
Current Price: $63735.39 USD
Day Change: -2161.22 (-3.28%)
Week Change: -3922.62 (-5.80%)
Month Change: -14930.85 (-18.98%)
24h Range: $63081.90 - $68162.47
Year Change: -27.19%
Volatility Context: Day High $66034.54 / Low $63081.90


## 3. Implement Parallel Agents

Notice that `MarketDataProvider` and `BinanceTracker` don't depend on each other's output.

In [4]:
from vinagent.multi_agent import AgentNode
from vinagent.logger.logger import logging_message

class MarketDataProvider(AgentNode):
    """Fetches and analyzes stock market data."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        # In a real scenario, this would use fetch_stock_data tool
        prompt = "Analyze FPT's current market sentiment and technical outlook. Provide a summary for a trader."
        output = self.invoke(prompt)
        return {
            "messages": {"role": "MarketDataProvider", "content": output.content if hasattr(output, "content") else str(output)},
            "equity_insight": output.content if hasattr(output, "content") else str(output)
        }

class BinanceTracker(AgentNode):
    """Fetches and analyzes crypto market data."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        # Instruction to use the dedicated BTC tool for performance metrics
        prompt = (
            "Analyze Bitcoin (BTC-USD) current performance, trends, and volatility. "
            "IMPORTANT: Use the 'fetch_btc_market_data' tool to get comprehensive metrics "
            "(daily/weekly/monthly changes and ranges). Provide a professional summary for a trader."
        )
        output = self.invoke(prompt)
        return {
            "messages": {"role": "BinanceTracker", "content": output.content if hasattr(output, "content") else str(output)},
            "crypto_insight": output.content if hasattr(output, "content") else str(output)
        }

class RecommenderAgent(AgentNode):
    """Merges insights into a unified strategy."""
    @logging_message
    def exec(self, state: TradingState) -> dict:
        equity = state.get("equity_insight", "")
        crypto = state.get("crypto_insight", "")
        
        prompt = (
            f"Equity Analysis: \n{equity}\n\n"
            f"Crypto Analysis: \n{crypto}\n\n"
            f"Based on both markets, provide a unified multi-asset trading recommendation."
        )
        output = self.invoke(prompt)
        return {
            "messages": {"role": "Recommender", "content": output.content if hasattr(output, "content") else str(output)},
            "final_recommendation": output.content if hasattr(output, "content") else str(output)
        }


## 4. Assemble the Parallel Flow

We use the `START` constant to trigger multiple agents simultaneously.

In [5]:
from customize_tools import fetch_stock_data_vn
from vinagent.multi_agent import CrewAgent
from vinagent.graph.operator import FlowStateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

instr = "CRITICAL: Format tool arguments as JSON object. NEVER use plain strings."

market_agent = MarketDataProvider(
    name="market_agent", 
    llm=llm, 
    tools=[],
    instruction=instr
)

crypto_agent = BinanceTracker(
    name="crypto_agent", 
    llm=llm, 
    tools=['vinagent.tools.websearch_tools'], 
    instruction=instr
)

# 1. FIX: Give the recommender a specific instruction so it stops trying to act like a data agent
recommender = RecommenderAgent(
    name="recommender", 
    llm=llm, 
    tools=[], 
    instruction="You are a Portfolio Manager. Synthesize the provided market insights into a final recommendation. DO NOT fetch new data or use tools."
)

global_tools = [
    get_current_time,
    fetch_btc_market_data,
    primary_function(fetch_stock_data_vn)
]

# 2. FIX: Register tools to ALL agents as a safety net to prevent NoneType crashes
for agent in [market_agent, crypto_agent, recommender]:
    for tool in global_tools:
        agent.tools_manager.register_function_tool(tool)

print("Agents instantiated with standardized tool registration.")

# 3. Build the crew
crew = CrewAgent(
    llm=llm,
    checkpoint=MemorySaver(),
    graph=FlowStateGraph(TradingState),
    flow=[
        START >> market_agent,
        START >> crypto_agent,
        market_agent >> recommender,
        crypto_agent >> recommender,
        recommender >> END
    ]
)
print("Phase 2 Crew Assembled with Direct Parallel Flow.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.register.tool:Registered search_api:
{'tool_name': 'search_api', 'arguments': 'query (Union[str, dict[str, str]]): The input query to search', 'return': 'The answer from search query', 'docstring': 'Search for an answer from a query string\n    Args:\n        query (dict[str, str]):  The input query to search\n    Returns:\n        The answer from search query', 'dependencies': ['os', 'dotenv', 'tavily', 'dataclasses', 'typing', 'vinagent'], 'module_path': 'vinagent.tools.websearch_tools', 'tool_type': 'module', 'tool_call_id': 'tool_5d2bbf51-27c4-4639-944e-3ab5b7e78df'}
INFO:vinagent.register.tool:Completed registration for module vinagent.tools.websearch_tools
INFO:vinagent.register.tool:Registered tool: get_current_time (runtime)
INFO:vinagent.register.tool:Registered tool: fetch_btc_market_data (runtime)
INFO:vinagent.register.tool:Registered tool: fetch_stock_data_vn (runtime)


Agents instantiated with standardized tool registration.
Phase 2 Crew Assembled with Direct Parallel Flow.


## 5. Execute Multi-Market Analysis

Run the system to see parallel processing in action.

In [6]:
from IPython.display import display, Markdown

query = "Compare FPT and BTC performance for this week and suggest a 70/30 split strategy."

result = crew.invoke(
    query=query,
    user_id="analyst",
    thread_id=23
)

display(Markdown(f"""
### 📈 Multi-Asset Trading Recommendation
---
{result['final_recommendation']}
"""))


INFO:vinagent.multi_agent.crew:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:No authentication card provided, skipping authentication
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:vinagent.agent.agent:Tool calling iteration 1/10


{'input': {'messages': {'role': 'user', 'content': 'Compare FPT and BTC performance for this week and suggest a 70/30 split strategy.'}}, 'config': {'configurable': {'user_id': 'analyst'}, 'thread_id': 23}}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:Executing tool call: {'tool_name': 'fetch_btc_market_data', 'tool_type': 'function', 'arguments': {}, 'module_path': '__runtime__'}
INFO:vinagent.register.tool:Completed executing function tool fetch_btc_market_data({})
INFO:vinagent.agent.agent:Tool calling iteration 2/10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.agent.agent:No more tool calls needed. Completed in 2 iterations.
INFO:vinagent.logger.logger:

============ BinanceTracker Response ============
{'messages': {'role': 'BinanceTracker', 'content': 'Bitcoin (BTC-USD) is currently trading at $63,737.68, reflecting a daily decrease of $2,158.93, which is a 3.28% drop. Over the past week, Bitcoin has seen a decline of $3,920.33, or 5.79%. Monthly performance shows a significant decrease of $14,928.56, equating to an 18.98% drop.\n\nThe 24-hour trading r


### 📈 Multi-Asset Trading Recommendation
---
Based on the provided market insights, the current outlook for Bitcoin (BTC-USD) indicates significant volatility and a downtrend. Bitcoin is trading at $63,737.68, witnessing a daily drop of 3.28% and a year-to-date decline of 27.19%. This environment presents substantial risks due to the price fluctuations. 

In light of these factors, the recommendation is to approach Bitcoin with caution. If unknown_user is considering investments in Bitcoin, it may be prudent to either hold existing positions while monitoring market conditions or to look for potential short-selling opportunities during the periods of heightened volatility. Alternative asset classes or traditional equities may offer a more stable investment path under current market conditions. 

Diversifying the portfolio could help mitigate risks associated with the current volatility in the cryptocurrency market.
